In [30]:
import json
import itertools
import re
import datetime
from difflib import SequenceMatcher
import pandas as pd
import numpy as np

In [31]:
#gets everything combined in list [0,1,2,3] stat,sherdog,tapology,ranking
combined = json.load(open('combined_dictionary_June_3','r'))

In [32]:
#dict of card names into dates
dict_of_cards_dates = json.load(open('ufc_stats_event_dates_june_3','r'))
no_dot_dict = {k.replace('.',''):v for k,v in dict_of_cards_dates.items() if '.' in k}
dot_dict = {re.sub(' vs ',' vs. ',k):v for k,v in dict_of_cards_dates.items() if re.search(' vs ',k)}
dict_of_cards_dates.update(no_dot_dict)
dict_of_cards_dates.update(dot_dict)

In [33]:
tapology_to_ufcstats = {'Alberto Cerra Leon':'Alberta Cerra Leon',
 'Aleksandra Albu':'Alexandra Albu',
 'Alex Ricci':'Alessandro Ricci',
 'Alexey Kunchenko':'Aleksei Kunchenko',
 'Alexey Oleinik':'Aleksei Oleinik',
 'Allen Berubie':'Allen Berube',
 'Alp Ozkilic':'Alptekin Ozkilic',
 'Alex da Silva':'Alex Da Silva',
 'Alexander Torres':'Alex Torres',
 'Anderson dos Santos':'Anderson Dos Santos',
 'Andrey Semenov':'Andrei Semenov',
 'Antonio dos Santos Jr.':'Antonio Dos Santos',
 'Anying Wang':'An Ying Wang',
 'Bharat Khandare':'Bharat Kandare',
 'Brad Scott':'Bradley Scott',
 'Bruno Mesquita':'Bruno Korea',
 'C.B. Dollaway':'CB Dollaway',
 'Carls John de Tomas':'Carls John De Tomas',
 'Chan Mi Jeon':'Chanmi Jeon',
 'Chris De La Rocha':'Chris de la Rocha',
 'Cristophe Leninger':'Christophe Leninger',
 'Cristiane Justino':'Cris Cyborg',
 'Dan Spohn':'Daniel Spohn',
 'Daniel Barrera':'Dan Barrera',
 'Daniel Hooker':'Dan Hooker',
 'Dave Kaplan':'David Kaplan',
 'Des Green':'Desmond Green',
 'Dmitrii Smolyakov':'Dmitrii Smoliakov',
 'Dmitriy Sosnovskiy':'Dmitry Sosnovskiy',
 'Dmitriy Stepanov':'Dmitri Stepanov',
 'Doo Ho Choi':'Dooho Choi',
 'Douglas Evans':'Doug Evans',
 'Edwin Dewees':'Edwin DeWees',
 'Eldo Dias Xavier':'Eldo Xavier Dias',
 'Elizeu Zaleski':'Elizeu Zaleski dos Santos',
 'Emil Weber Meek':'Emil Meek',
 'Emily  Kagan':'Emily Peters Kagan',
 'Felix Mitchell':'Felix Lee Mitchell',
 "Francis N'Gannou":'Francis Ngannou',
 'Francisco Rivera Jr.':'Francisco Rivera',
 'Georges St. Pierre':'Georges St-Pierre',
 'Heather Clark':'Heather Jo Clark',
 'Guilherme Bomba':'Guilherme Vasconcelos',
 'Jim Wallhead':'Jimmy Wallhead',
 'Humberto  Brown Morrison':'Humberto Brown',
 'Joao Marcos Pierini':'Joao Pierini',
 'Joey Merritt':'Joe Merritt',
 'John Campatella':'John Campetella',
 'John Macapa':'John Teixeira',
 'John Olav Einemo':'Jon Olav Einemo',
 'Jose Maria Tome':'Jose Maria',
 'Josh Burkman':'Joshua Burkman',
 'Josh Sampo':'Joshua Sampo',
 'Jose Alberto Quinones':'Jose Quinonez',
 'Josh Schockman':'Josh Shockman',
 'Juan Manuel Puig':'Juan Puig',
 'Junior dos Santos':'Junior Dos Santos',
 'Khalil Rountree':'Khalil Rountree Jr.',
 'Kris Rotharmel':'Kristian Rothaermel',
 'LaVerne Clark':'Laverne Clark',
 'Lance Gibson Sr.':'Lance Gibson',
 'Livia Renata Souza':'Livinha Souza',
 'Luiz Dutra Jr.':'Luiz Dutra',
 'Maia Kahaunaele-Stevenson':'Maia Stevenson',
 'Manny Gamburyan':'Manvel Gamburyan',
 'Marcio Alexandre Jr.':'Marcio Alexandre Junior',
 'Kimbo Slice':'Kevin Ferguson',
 'Marco Antonio Beltran':'Marco Beltran',
 'Matt Riddle':'Matthew Riddle',
 'Mike Patt':'Michael Patt',
 'Mike Trizano':'Michael Trizano',
 'Mike Graves':'Michael Graves',
 'Mike De La Torre':'Mike de la Torre',
 'Mirko Filipovic':'Mirko Cro Cop',
 'Antonio Rodrigo Nogueira':'Minotauro Nogueira',
 'Mostapha Al Turk':'Mostapha Al-Turk',
 'Nah-shon Burrell':'Nah-Shon Burrell',
 'Nick Negumereanu':'Nicolae Negumereanu',
 'Orlando Weit':'Orlando Wiet',
 'Ovince St. Preux':'Ovince Saint Preux',
 'Pat Walsh':'Patrick Walsh',
 'Phil De Fries':'Philip De Fries',
 'Quinton Jackson':'Rampage Jackson',
 'Rafael dos Anjos':'Rafael Dos Anjos',
 'Rich Crunkilton':'Richard Crunkilton',
 'Rafael Cavalcante':'Rafael Feijao',
 'Robbie Peralta':'Robert Peralta',
 'Antonio Rogerio Nogueira':'Rogerio Nogueira',
 'Ron Van Clief':'Ron van Clief',
 'Rodolfo Rubio Perez':'Rodolfo Rubio',
 'Sam Morgan':'Sammy Morgan',
 'Ronaldo Souza':'Jacare Souza',
 'Sergey Pavlovich':'Sergei Pavlovich',
 'Sergey Spivak':'Serghei Spivac',
 'Seung Woo Choi':'Seungwoo Choi',
 'Shane Del Rosario':'Shane del Rosario',
 'Steven Kennedy':'Steve Kennedy',
 'Steven Lynch':'Stevie Lynch',
 'T.J. Dillashaw':'TJ Dillashaw',
 "T.J. O'Brien":"TJ O'Brien",
 'Thomas Speer':'Tommy Speer',
 'Timothy Johnson':'Tim Johnson',
 'Tsuyoshi Kosaka':'Tsuyoshi Kohsaka',
 'William Macario':'William Patolino',
 'Xiaonan Yan':'Yan Xiaonan',
 'Yadong Song':'Song Yadong',
 'Yanan Wu':'Wu Yanan',
 'Yaozong Hu':'Hu Yaozong',
 'Seo Hee Ham':'Seohee Ham',
 'Tony Christodoulou':'Anthony Christodoulou',
 'Douglas Andrade':'Douglas Silva de Andrade',
 'Luis Henrique da Silva':'Henrique da Silva',
 'Jian Ping Yang':'Jianping Yang',
 'Edimilson Souza':'Kevin Souza',
 'Guan Wang':'Wang Guan',
 'Lipeng Zhang':'Zhang Lipeng',
 'Azunna Anyanwu':'Zu Anyanwu',
 'Zhikui Yao':'Yao Zhikui',
 'War Machine':'Jon Koppenhaver',
 'Kenan Song':'Song Kenan',
 'Ronald Stallings':'Ron Stallings',
 'Roli Delgado':'Rolando Delgado',
 'Robert Whiteford':'Rob Whiteford',
 'Guangyou Ning':'Ning Guangyou',
 'Mike Van Arsdale':'Mike van Arsdale',
 'Ramiro Hernandez':'Junior Hernandez',
 'Leonardo Guimaraes':'Leonardo Augusto Leleco',
 'Jingliang Li':'Li Jingliang',
 'Pingyuan Liu':'Liu Pingyuan',
 'Tuerxun Jumabieke':'Jumabieke Tuerxun',
 'Kazuo Takahashi':'Yoshiki Takahashi'}
ufcstats_to_tapology = {v: k for k, v in tapology_to_ufcstats.items()}

In [34]:
sherdog_to_ufcstats = {'Alan Patrick Silva Alves':'Alan Patrick',
 'Alberto Cerra Leon':'Alberta Cerra Leon',
 'Alexey Oleynik':'Aleksei Oleinik',
 'Alex da Silva Coelho':'Alex Da Silva',
 'Amanda Bobby Cooper':'Amanda Cooper',
 'Aleksandra Albu':'Alexandra Albu',
 'Anderson dos Santos':'Anderson Dos Santos',
 'Alexey Kunchenko':'Aleksei Kunchenko',
 'Antonio Carlos Jr.':'Antonio Carlos Junior',
 'Antonio dos Santos Jr.':'Antonio Dos Santos',
 'Anying Wang':'An Ying Wang',
 'B.J. Penn':'BJ Penn',
 'Arjan Singh Bhullar':'Arjan Bhullar',
 'Anistavio Medeiros de Figueiredo':'Anistavio Medeiros',
 'Alberto Emiliano Pereira':'Alberto Uda',
 'Ben Alloway':'Benny Alloway',
 'Bharat Khandare':'Bharat Kandare',
 'C.B. Dollaway':'CB Dollaway',
 'Bruno Rodrigues Mesquita':'Bruno Korea',
 'C.J. Keith':'CJ Keith',
 'Carlo Pedersoli Jr.':'Carlo Pedersoli',
 'Carls John de Tomas':'Carls John De Tomas',
 'Cesar Marcussi':'Cesar Marscucci',
 'Claudio Henrique da Silva':'Claudio Silva',
 'Dan Spohn':'Daniel Spohn',
 'Daniel Barrera':'Dan Barrera',
 'Daniel Downes':'Danny Downes',
 'Daniel Stittgen':'Dan Stittgen',
 'Dave Roberts':'David Roberts',
 'Darrell Gholar':'Darrel Gholar',
 'Carlos Diego Ferreira':'Diego Ferreira',
 'Dieuseul Berto':'Dieusel Berto',
 'Dmitriy Sosnovskiy':'Dmitry Sosnovskiy',
 'Dmitry Smoliakov':'Dmitrii Smoliakov',
 'Doo Ho Choi':'Dooho Choi',
 'Douglas Evans':'Doug Evans',
 'Duda Santanna':'Duda Santana',
 'Edwin Dewees':'Edwin DeWees',
 'Emily Kagan':'Emily Peters Kagan',
 'Eldo Dias Xavier':'Eldo Xavier Dias',
 'Felipe Dias Colares':'Felipe Colares',
 'Georges St. Pierre':'Georges St-Pierre',
 'Felix Mitchell':'Felix Lee Mitchell',
 'Enrique Briones':'Henry Briones',
 'Godofredo Castro':'Godofredo Pepey',
 'Geraldo de Freitas Jr.':'Geraldo de Freitas',
 'He-Man Ali Gipson':'He-Man Gipson',
 'Henrique     da Silva':'Henrique da Silva',
 'J.C. Cottrell':'JC Cottrell',
 'J.J. Aldrich':'JJ Aldrich',
 'Jack McLaughlin':'Jack McGlaughlin',
 'Jim Crute':'Jimmy Crute',
 'Jim Hettes':'Jimy Hettes',
 'Jim Wallhead':'Jimmy Wallhead',
 'Joao Marcos Pierini':'Joao Pierini',
 'John Olav Einemo':'Jon Olav Einemo',
 'Jose Alberto Quinonez':'Jose Quinonez',
 'Jose Maria Tome':'Jose Maria',
 'John Teixeira da Conceicao':'John Teixeira',
 'Jon delos Reyes':'Jon Delos Reyes',
 'Jorge Antonio Cezario de Oliveira':'Jorge de Oliveira',
 'Josh Sampo':'Joshua Sampo',
 'Josh Schockman':'Josh Shockman',
 'Joseph Duffy':'Joe Duffy',
 'Juan Francisco Espino Dieppa':'Juan Espino',
 'Juan Manuel Puig':'Juan Puig',
 'Junie Allen Browning':'Junie Browning',
 'Junior dos Santos':'Junior Dos Santos',
 'K.J. Noons':'KJ Noons',
 'Khalil Rountree':'Khalil Rountree Jr.',
 'LaVerne Clark':'Laverne Clark',
 'Leonardo Mafra Texeira':'Leonardo Mafra',
 'Luis Henrique Barbosa de Oliveira':'Luis Henrique',
 'Luiz Jorge Dutra Jr.':'Luiz Dutra',
 'Maia Kahaunaele-Stevenson':'Maia Stevenson',
 'Manny Gamburyan':'Manvel Gamburyan',
 'Marcio Alexandre Jr.':'Marcio Alexandre Junior',
 'Marcos Vinicius Borges Pancini':'Marcos Vinicius',
 'Matheus Nicolau Pereira':'Matheus Nicolau',
 'Matt Riddle':'Matthew Riddle',
 'Michael Guymon':'Mike Guymon',
 'Mike Thomas Brown':'Mike Brown',
 'Mike Van Arsdale':'Mike van Arsdale',
 'Mirko Filipovic':'Mirko Cro Cop',
 'Antonio Rogerio Nogueira':'Rogerio Nogueira',
 'Antonio Rodrigo Nogueira':'Minotauro Nogueira',
 'Nicholas Musoke':'Nico Musoke',
 'Ovince St. Preux':'Ovince Saint Preux',
 'Paulo Henrique Costa':'Paulo Costa',
 'Rafael dos Anjos':'Rafael Dos Anjos',
 'Robert Whiteford':'Rob Whiteford',
 'Rich Crunkilton':'Richard Crunkilton',
 'Robert Emerson':'Rob Emerson',
 'Renato Carneiro':'Renato Moicano',
 'Robbie Peralta':'Robert Peralta',
 'Ronaldo Souza':'Jacare Souza',
 'Rodrigo Goiana de Lima':'Rodrigo de Lima',
 'Roli Delgado':'Rolando Delgado',
 'Rony Mariano Bezerra':'Rony Jason',
 'Sako Chivitchyan':'Sako Chivitchian',
 'Sam Morgan':'Sammy Morgan',
 'Seo Hee Ham':'Seohee Ham',
 'Sergey Spivak':'Serghei Spivac',
 'Steven Lynch':'Stevie Lynch',
 'Steven Ray':'Stevie Ray',
 'Sung Woo Choi':'Seungwoo Choi',
 'T.J. Dillashaw':'TJ Dillashaw',
 'T.J. Grant':'TJ Grant',
 "T.J. O'Brien":"TJ O'Brien",
 'T.J. Waldburger':'TJ Waldburger',
 'Thiago de Oliveira Perpetuo':'Thiago Perpetuo',
 'Tiago dos Santos e Silva':'Tiago Trator',
 'Timothy Johnson':'Tim Johnson',
 'Tom Speer':'Tommy Speer',
 'Tsuyoshi Kosaka':'Tsuyoshi Kohsaka',
 'Wendell de Oliveira Marques':'Wendell Oliveira',
 'Xiaonan Yan':'Yan Xiaonan',
 'Yadong Song':'Song Yadong',
 'Yanan Wu':'Wu Yanan',
 'Saparbek Safarov':'Saparbeg Safarov',
 'Vinicius Kappke de Queiroz':'Vinicius Queiroz',
 'Yuta Sasaki':'Ulka Sasaki',
 'Quinton Jackson':'Rampage Jackson',
 'Phil Brooks':'CM Punk',
 'Cristiane Justino':'Cris Cyborg',
 'Leonardo Guimaraes':'Leonardo Augusto Leleco',
 'Jussier da Silva':'Jussier Formiga',
 'Carlos Augusto Inocente Filho':'Guto Inocente',
 'Guan Wang':'Wang Guan',
 'Patrick Holohan':'Paddy Holohan',
 'Kenan Song':'Song Kenan',
 'Azunna Anyanwu':'Zu Anyanwu',
 'Kazuo Takahashi':'Yoshiki Takahashi',
 'William Macario':'William Patolino',
 'Marco Polo Reyes':'Polo Reyes',
 'Wagner Silva Gomes':'Wagner Silva',
 'Edimilson Souza':'Kevin Souza',
 'Pingyuan Liu':'Liu Pingyuan',
 'Rafael Cavalcante':'Rafael Feijao',
 'Ramiro Hernandez':'Junior Hernandez',
 'Yaozong Hu':'Hu Yaozong',
 'Heather Clark':'Heather Jo Clark',
 'Jingliang Li':'Li Jingliang',
 'Guangyou Ning':'Ning Guangyou',
 'Zhikui Yao':'Yao Zhikui',
 'Lipeng Zhang':'Zhang Lipeng',
 'War Machine':'Jon Koppenhaver',
 'Benjamin Duffy':'James Giboo'}

ufcstats_to_sherdog = {v: k for k, v in sherdog_to_ufcstats.items()}

In [35]:
rankings_to_ufc_stats = {'Jussier Da Silva':'Jussier Formiga',
 'Constantinos Philippou':'Costas Philippou',
 'Antonio Rogerio Nogueira':'Rogerio Nogueira',
 'Antonio Rodrigo Nogueira':'Minotauro Nogueira',
 'Ronaldo Souza':'Jacare Souza',
 'T.J. Dillashaw':'TJ Dillashaw',
 'Rafael Cavalcante':'Rafael Feijao',
 'Bigfoot Silva':'Antonio Silva',
 'Heather Clark':"Heather Jo Clark",
 'Jessica Rose Clark':"Jessica-Rose Clark",
 'Elizeu dos Santos':"'Elizeu Zaleski dos Santos'",
 'Kai Kara France':"Kai Kara-France",
 'Casey  Kenney':'Casey Kenney'}
ufcstats_to_rankings = {v: k for k, v in rankings_to_ufc_stats.items()}

In [36]:
women_dict = json.load(open("womens dict",'r'))
women_dict['Alfonso Alcarez'] = 0
women_dict['CJ Fernandes'] = 0
women_dict['Mark Robinson'] = 0
women_dict['Ron Faircloth'] = 0
women_dict['Pedro Nobre'] = 0

In [37]:
def close_dates(date1,date2):
    if issubclass(type(date1),datetime.date) and issubclass(type(date2),datetime.date):
        return abs((date1 - date2).days) < 8
    fight1 = datetime.date(*(int(y) for y in date1.split('-')))
    fight2 = datetime.date(*(int(y) for y in date2.split('-')))
    return abs((fight1 - fight2).days) < 4

In [38]:
def to_date(date):
    return datetime.date(*(int(y) for y in date.split('-')))

In [39]:
def get_biggest_slice(numbers_list):
    if len(numbers_list) == 1:return numbers_list
    big_list = []
    for idx,x in enumerate(range(max(numbers_list),min(numbers_list)-1,-1)):
        if sorted(numbers_list,reverse=True)[idx] == x:
            big_list.append(x)
        else:
            return big_list
    return big_list

In [40]:
def find_closest_numbers(val,check_list):
    val_list = []
    for x in range(1,6):
        val_list.append(val+x)
        val_list.append(val-x)
    val_list = [x for x in val_list if x >= 0]
    for x in val_list:
        if x in check_list:
            return x
    return False

In [41]:
def sorting_with_None(x):
    if x[1]!=None and x[0]!=None:
        return (x[1]+x[0])/2
    elif x[1]!=None:
        return x[1]-0.6
    return x[0]-0.6

In [42]:
def replace_close_nones(combinations,indexes_list):
    for x in indexes_list:
        a,b = combinations[x[0]], combinations[x[1]]
        del combinations[x[1]]
        del combinations[x[0]]
        if a[0]:
            ab = [a[0],b[1]]
        else:
            ab = [a[1],b[0]]
        combinations.append(ab)
        return combinations

In [43]:
def combine_bouts(sher_bouts,tap_bouts):
    combinations = []
    
    sher_length = len(sher_bouts)
    tap_length = len(tap_bouts)
    
    unused_sher,unused_tap = list(range(sher_length)), list(range(tap_length))
    
    def check_for_identical():
        to_delete = []
        for x in unused_sher:
            for y in unused_tap:
                if x == y:
                    combinations.append([x,y])
                    to_delete.append(x)
        
        for x in to_delete:
            unused_sher.remove(x)
            unused_tap.remove(x)
        
    
    def remove_used_indexes():
        nonlocal unused_sher
        nonlocal unused_tap

        unused_sher = list(set(unused_sher) - set([x[0] for x in combinations]))
        unused_tap = list(set(unused_tap) - set([x[1] for x in combinations]))
    
    
    
    used_idx,used_idy = [],[]
    for x in unused_sher:
        for y in unused_tap:
            if (sher_bouts[x]['date'] and tap_bouts[y]['fight date']) and (x not in used_idx and y not in used_idy):
                if close_dates(sher_bouts[x]['date'],tap_bouts[y]['fight date']) and sher_bouts[x]['opponent'] == tap_bouts[y]["Opponent's name"]:
                    combinations.append([x,y])
                    used_idx.append(x)
                    used_idy.append(y)
    
    remove_used_indexes()
    #check_for_identical()
    
    for x in unused_sher:
        for y in unused_tap:
            if sher_bouts[x]['date'] and tap_bouts[y]['fight date'] and (x not in used_idx and y not in used_idy):
                if close_dates(sher_bouts[x]['date'],tap_bouts[y]['fight date']):
                    combinations.append([x,y])
                    used_idx.append(x)
                    used_idy.append(y)
                    break
    
    remove_used_indexes()
    
    
    for x in sorted(unused_sher):
        for y in sorted(unused_tap):
            if sher_bouts[x]['opponent'] == tap_bouts[y]["Opponent's name"] and (x not in used_idx and y not in used_idy):
                combinations.append([x,y])
                used_idx.append(x)
                used_idy.append(y)
                break
    
    remove_used_indexes()
    
    for x in sorted(unused_sher):
        for y in sorted(unused_tap):
            if (SequenceMatcher(None,sher_bouts[x]['opponent'],tap_bouts[y]["Opponent's name"]).ratio() > 0.8) and (x not in used_idx and y not in used_idy):
                combinations.append([x,y])
                used_idx.append(x)
                used_idy.append(y)
                break
    
    remove_used_indexes()
    cleaning = False
    #approximation combinning
    if unused_sher and unused_tap:
        for x in sorted(unused_sher):
            y_approx =  find_closest_numbers(x,unused_tap)
            if y_approx:
                combinations.append([x,y_approx])
                used_idx.append(x)
                used_idy.append(y_approx) 
    
        remove_used_indexes()
        cleaning = True
    combinations = sorted(combinations)
    
    if unused_sher:
        for x in unused_sher:
            combinations.append([x,None])
            used_idx.append(x)
            cleaning = True
    
    if unused_tap:
        for y in unused_tap:
            combinations.append([None,y])
            used_idy.append(y)
            cleaning = True
    
    combinations = sorted(combinations,key=sorting_with_None)
    
    concat_idx = []
    for idx,x in enumerate(combinations):
        if x[1] == None:
            if (idx+1<len(combinations)) and (combinations[idx+1][0]==None):
                concat_idx.append([idx,idx+1])
        elif x[0]==None:
            if (idx+1<len(combinations)) and (combinations[idx+1][1]==None):
                concat_idx.append([idx,idx+1])
        cleaning = True
    
    if cleaning and concat_idx:
        combinations = replace_close_nones(combinations,concat_idx)
    
    
    
    return combinations

In [44]:
def get_ranking_date(date:str,rankings:dict) -> list:
    date = to_date(date)
    keys = sorted([x for x in rankings.keys()])
    dates = [to_date(x) for x in rankings.keys()]
    dates.insert(0,date)
    dates.sort()
    idx = dates.index(date)
    if idx==0:
        if (to_date(keys[0]) - date).days > 90:
            return [None,None]
        return [None,rankings[keys[0]]]
    else:
        if idx < len(keys):
            return [rankings[keys[idx-1]],rankings[keys[idx]]]
        else:
            return [rankings[keys[idx-1]],None]

In [45]:
def get_ranking_position(ufc,rankings):
    fights = []
    if rankings: 
        for x in ufc['bouts']:
            card_name = x['Card name']
            date = dict_of_cards_dates[card_name]
            fights.append([x,date,get_ranking_date(date,rankings)])
        return fights
    else:
        return [[x , dict_of_cards_dates[x['Card name']], [None,None] ]  for x in ufc['bouts']]
    

In [46]:
def assembling(fighter):
    ufc_stats = fighter[0]
    sher = fighter[1]
    tap = fighter[2]
    rankings = fighter[3]
    fighter_bouts = []
    stats_list = get_ranking_position(fighter[0],fighter[3])
    if len(sher['bouts']) == len(tap['bouts']):
        for x in zip(sher['bouts'],tap['bouts']):
            fighter_bouts.append([x[0],x[1]])
    else:
        for x in combine_bouts(sher['bouts'],tap['bouts']):
            if x[0]!=None and x[1]!=None:
                fighter_bouts.append([ sher['bouts'][x[0]], tap['bouts'][x[1]] ])
            elif x[0]!=None:
                fighter_bouts.append([ sher['bouts'][x[0]], None ])
            else:
                fighter_bouts.append([ None , tap['bouts'][x[1]] ])
    
    for x in fighter_bouts:
        for y in stats_list:
            if x[0] and close_dates(x[0]['date'], y[1]):
                x += y
                break
    
    for x in fighter_bouts:
        if len(x) != 5:
            x += [{},None,[None,None]] 
    
    fighter_information = [{k:i for k,i in ufc_stats.items() if k!='bouts'},
                           {k:i for k,i in sher.items() if k!='bouts'},
                           {k:i for k,i in tap.items() if k!='bouts'},
                          fighter_bouts]
    return fighter_information

In [47]:
everything = []

In [48]:
for idx,x in enumerate(combined):
    print(idx,end=' ')
    everything.append(assembling(x))

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [49]:
import csv

In [50]:
def ufcstat_profile_to_string(stat_dict):
    Names = ["Name",'Woman',"Nickname","Height in","Reach in","Stance","Sig Land pm","Strik acc","Sig Absorb pm","Strik def","TD Avg","TD acc","TD Def","Sub Avg"]
    string = []
    string+= [stat_dict['Name']]
    string+= [str(women_dict[stat_dict['Name']])]
    string+= [stat_dict['Nickname']] 
    string+= [stat_dict['Height']]
    string+= [stat_dict['Reach']]
    string+= [stat_dict['STANCE']] 
    string+= [stat_dict['Carrer stats']['SLpM']] 
    string+= [stat_dict['Carrer stats']['Str. Acc.']] 
    string+= [stat_dict['Carrer stats']['SApM']]
    string+= [stat_dict['Carrer stats']['Str. Def']] 
    string+= [stat_dict['Carrer stats']['TD Avg.']]
    string+= [stat_dict['Carrer stats']['TD Acc.']] 
    string+= [stat_dict['Carrer stats']['TD Def.']]
    string+= [stat_dict['Carrer stats']['Sub. Avg.']]
    
    return(Names,string)

In [51]:
def sher_profile_to_string(stat_dict):
    Names = ["DoB","Height cm","Wins","Loses","draws","nc","Affil","National","Loc reg","Loc city","W KO","W sub","W dec","W oth","L KO","L sub","L dec","L oth"]
    
    string = []
    string+= [stat_dict['Birth date'] ]
    string+= [stat_dict['Height']]
    string+= [stat_dict['Record']['wins']]
    string+= [stat_dict['Record']['loses']] 
    string+= [stat_dict['Record']['draws']] 
    string+= [stat_dict['Record']['nc']] 
    
    string+= [stat_dict['Affiliation']] 
    string+= [stat_dict['Nationality']]
    string+= [stat_dict['Location region'].strip()]
    string+= [stat_dict['Location city'].strip()] 
    
    string+= [stat_dict['wins summary']['KO/TKO']] 
    string+= [stat_dict['wins summary']['SUBMISSIONS']] 
    string+= [stat_dict['wins summary']['DECISIONS']] 
    string+= [stat_dict['wins summary']['OTHERS']]
    
    string+= [stat_dict['loses summary']['KO/TKO']] 
    string+= [stat_dict['loses summary']['SUBMISSIONS']]
    string+= [stat_dict['loses summary']['DECISIONS']]
    string+= [stat_dict['loses summary']['OTHERS']] 
    
    return (Names,string)

In [52]:
def tap_profile_to_string(stat_dict):
    Names = ["Giv name","Reach cm","Height cm","Last Fight","Born","Fight out of"]
    string = []
    string+= [stat_dict['Given name'].replace(',',' ')]
    string+= [stat_dict['Reach']]
    string+= [stat_dict['Height']]
    
    string+= [stat_dict['Last fight']]
    string+= [stat_dict['Born'].replace(',',' ')] 
    string+= [stat_dict['Fighting out of'].replace(',',' ')]
    
    return (Names,string)

In [53]:
def get_weightclass_place(listOfDicts):
    dict_weights = {'Pound-for-Pound':0,'Flyweight':1,'Bantamweight':2,'Featherweight':3,'Lightweight':4,'Welterweight':5,
                     "Middleweight":6,'Light Heavyweight':7,'Heavyweight':8,"Women's Strawweight":9,"Women's Flyweight":10,"Women's Bantamweight":11,"Women's Featherweight":12}
    
    list_before = ['']*13
    list_after = ['']*13
    
    if listOfDicts[0]:
        for x in listOfDicts[0].keys():
            list_before[dict_weights[x]] = str(round(listOfDicts[0][x]))+','
    
    if listOfDicts[1]:
        for x in listOfDicts[1].keys():
            list_after[dict_weights[x]] = str(round(listOfDicts[1][x]//1))+','
            
    return [list_before,list_after]

In [54]:
def bout_decomposition(bout,fighter_name):
    Names = ["fr wins","fr loses","fr draws","opponent","or wins","or loses","or draws","result","round","time","method","decision","injuries","weightclass","wcls weight"]
    Names += ["weigh-in","fighter title","fights title name","referee","fight time","billing","odds","event","pay","date","dc reason"]
    bout_string = []
    sher,tap,ufc_stat,date,rank = bout
    
    
    if tap:
        bout_string+= [tap["Fighter's record"]['wins']]
        bout_string+= [tap["Fighter's record"]['loses']]
        bout_string+= [tap["Fighter's record"]['draws']]
    else:
        bout_string+=['']*3
        
        
    if sher:
        bout_string+= [sher['opponent']] 
    else:
        bout_string+=['']

    if tap and tap["Opponent's record"]:
        bout_string+= [tap["Opponent's record"]['wins']]
        bout_string+= [tap["Opponent's record"]['loses']] 
        bout_string+= [tap["Opponent's record"]['draws']] 
    else:
        bout_string+=['']*3
    
    if sher:
        bout_string+= [ sher['result'] ]
        bout_string+= [ sher['round'] ]
        bout_string+= [ sher['time'] ]
        bout_string+= [ sher['method'] ]
    else:
        bout_string+= ['']*4
    
    if tap:
        bout_string+= [ tap['Decision'] ] 
        bout_string+= [ tap['Injuries'] ]    
        bout_string+= [ tap['Weightclass'] ]
        bout_string+= [ tap['Class weight'] ]
        bout_string+= [ tap['Weigh-In'] ] 
        bout_string+= [ tap['Title in a fight'] ]
        bout_string+= [ tap['Weightclass title name'] ]
        bout_string+= [ tap['Referee'] ]
        bout_string+= [ tap['Duration'] ]
        bout_string+= [ tap['Billing'] ]
        bout_string+= [ tap['Odds'] ]
    else:
        bout_string+=['']*11
    
    if sher:
        bout_string+= [ sher['event'] ]
    else:
        bout_string+=['']
    
    if tap:
        bout_string+= [ tap['Pay'] ]
    else:
        bout_string+=['']
    
    if sher:
        bout_string+= [ sher['date'] ]
        bout_string+= [ sher['dc reason'] ]
    else:
        bout_string+=['']*2
    

    if ufc_stat and len(ufc_stat['Fighter1 stats']['Totals per Round']) < 6:
        
        Names += ['UFC']
        bout_string += ['1']
        
        ufc_additional_names = ["fight bonus","perf bonus","sub bonus","ko bonus","off method","off round","title fight","details","off time","time format","off referee","f card1","op card1","f card2","op card2","f card3","op card3"]
        
        ufc_additional_string = []
        
        main_name = 'Fighter1' if ufc_stat['Fighter1 stats']['Totals']['Name'] == fighter_name else 'Fighter2'
        sub_name = 'Fighter2' if main_name == 'Fighter1' else 'Fighter1'
        
        ufc_additional_string += [str(ufc_stat['fight bonus'])] 
        ufc_additional_string += [str(ufc_stat['perf bonus'])] 
        ufc_additional_string += [str(ufc_stat['sub bonus'])] 
        ufc_additional_string += [str(ufc_stat['ko bonus'])]
        ufc_additional_string += [str(ufc_stat['Method'])]
        ufc_additional_string += [str(ufc_stat['Round'])]
        ufc_additional_string += [str(ufc_stat['title fight'])] 
        ufc_additional_string += [ufc_stat['Details'].strip()]
        
        ufc_additional_string += [ufc_stat['Time']]
        ufc_additional_string += [ufc_stat['Time format']]
        ufc_additional_string += [ufc_stat['Referee']]
               
        if ufc_stat['Score 1']: 
            ufc_additional_string += [ufc_stat['Score 1'][main_name]]
            ufc_additional_string += [ufc_stat['Score 1'][sub_name]]
            ufc_additional_string += [ufc_stat['Score 2'][main_name]]
            ufc_additional_string += [ufc_stat['Score 2'][sub_name]]
            ufc_additional_string += [ufc_stat['Score 3'][main_name]]
            ufc_additional_string += [ufc_stat['Score 3'][sub_name]]
        else:
            ufc_additional_string += ['']*6
        
        Names += ufc_additional_names
        bout_string += ufc_additional_string
        
        totals = ["tot f KD","tot f SIG STR att","tot f SIG STR lan","tot f SIG STR %","tot f TOTAL STR att",
                  "tot f TOTAL STR lan","tot f TD att","tot f TD lan","tot f TD %","tot f SUB ATT","tot f PASS","tot f REV"]
        signifs = ["sig f SIG STR att","sig f SIG STR lan","sig f SIG STR %","sig f HEAD att","sig f HEAD lan","sig f BODY att","sig f BODY lan",
                   "sig f LEG att","sig f LEG lan","sig f DISTANCE att","sig f DISTANCE lan","sig f CLINCH att","sig f CLINCH lan","sig f GROUND att","sig f GROUND lan"]
        
        fighterstotals = [[],[]]
        for idx,f_name in enumerate([f"{main_name} stats", f"{sub_name} stats"]):
            
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['KD']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['SIG. STR.']['attempted']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['SIG. STR.']['landed']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['SIG. STR. %']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['TOTAL STR.']['attempted']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['TOTAL STR.']['landed']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['TD']['attempted']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['TD']['landed']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['TD %']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['SUB. ATT']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['PASS']]
            fighterstotals[idx] += [ufc_stat[f_name]['Totals']['REV.']]
            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['SIG. STR.']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['SIG. STR.']['landed']]             
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']["SIG. STR. %"]]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['HEAD']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['HEAD']['landed']]             
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['BODY']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['BODY']['landed']]             
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['LEG']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['LEG']['landed']]             
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['DISTANCE']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['DISTANCE']['landed']]  
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['CLINCH']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['CLINCH']['landed']]  
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['GROUND']['attempted']]            
            fighterstotals[idx] += [ufc_stat[f_name]['Significant Strikes']['GROUND']['landed']]  
            
            if idx == 0:
                Names += totals
                Names += signifs
                bout_string += fighterstotals[idx]
            else:
                Names += [x.replace('f','o') for x in totals]
                Names += [x.replace('f','o') for x in signifs]
                bout_string += fighterstotals[idx]
            
            
            
        fighters1totals = [[],[],[],[],[]]
        fighters2totals = [[],[],[],[],[]]

        for x in range(1,6): 
            Names += [a + f' pr{x}' for a in signifs]

        for x in range(1,6): 
            Names += [a.replace('f','o') + f' pr{x}' for a in signifs]
        
        fought_rounds = 0
        
        
        for idx,f_name in enumerate([f"{main_name} stats",f"{sub_name} stats"]):
            for idy,round_ in enumerate(ufc_stat[f_name]['Significant Strikes per Round']):
                if idx == 0:

                    fighters1totals[idy] += [ round_['SIG. STR.']['attempted']]            
                    fighters1totals[idy] += [ round_['SIG. STR.']['landed']]             

                    fighters1totals[idy] += [ round_["SIG. STR. %"]]            

                    fighters1totals[idy] += [ round_['HEAD']['attempted']]            
                    fighters1totals[idy] += [ round_['HEAD']['landed']]             

                    fighters1totals[idy] += [ round_['BODY']['attempted']]            
                    fighters1totals[idy] += [ round_['BODY']['landed']]             

                    fighters1totals[idy] += [ round_['LEG']['attempted']]            
                    fighters1totals[idy] += [ round_['LEG']['landed']]             

                    fighters1totals[idy] += [ round_['DISTANCE']['attempted']]            
                    fighters1totals[idy] += [ round_['DISTANCE']['landed']]  

                    fighters1totals[idy] += [ round_['CLINCH']['attempted']]            
                    fighters1totals[idy] += [ round_['CLINCH']['landed']]  

                    fighters1totals[idy] += [ round_['GROUND']['attempted']]            
                    fighters1totals[idy] += [ round_['GROUND']['landed']]

                    bout_string += fighters1totals[idy]
                    fought_rounds = 1+idy


                if idx == 1:
                    if idx == 1 and idy == 0:
                        for a in range(fought_rounds+1,6):
                            bout_string += ['']*15

                    fighters2totals[idy] += [ round_['SIG. STR.']['attempted']]            
                    fighters2totals[idy] += [ round_['SIG. STR.']['landed']]             
                    fighters2totals[idy] += [ round_["SIG. STR. %"]]            
                    fighters2totals[idy] += [ round_['HEAD']['attempted']]            
                    fighters2totals[idy] += [ round_['HEAD']['landed']]             
                    fighters2totals[idy] += [ round_['BODY']['attempted']]            
                    fighters2totals[idy] += [ round_['BODY']['landed']]             
                    fighters2totals[idy] += [ round_['LEG']['attempted']]            
                    fighters2totals[idy] += [ round_['LEG']['landed']]             
                    fighters2totals[idy] += [ round_['DISTANCE']['attempted']]            
                    fighters2totals[idy] += [ round_['DISTANCE']['landed']]  
                    fighters2totals[idy] += [ round_['CLINCH']['attempted']]            
                    fighters2totals[idy] += [ round_['CLINCH']['landed']]  
                    fighters2totals[idy] += [ round_['GROUND']['attempted']]            
                    fighters2totals[idy] += [ round_['GROUND']['landed']]


                    bout_string += fighters2totals[idy]



        if fought_rounds == 0:
            for a in range(fought_rounds+1,6):
                bout_string += ['']*30
                
        else:
            for a in range(fought_rounds+1,6):
                bout_string += ['']*15
        

        fighters1totals = [[],[],[],[],[]]
        fighters2totals = [[],[],[],[],[]]

        
        for x in range(1,6): 
            Names += [a + f' pr{x}' for a in totals]

        for x in range(1,6): 
            Names += [a.replace('f','o') + f' pr{x}' for a in totals]


        
        for idx,f_name in enumerate([f"{main_name} stats",f"{sub_name} stats"]):
            for idy,round_ in enumerate(ufc_stat[f_name]['Totals per Round']):
                if idx == 0:

                    fighters1totals[idy] += [ round_['KD']]           
                    fighters1totals[idy] += [ round_['SIG. STR.']['attempted']]         
                    fighters1totals[idy] += [ round_['SIG. STR.']['landed']]    
                    fighters1totals[idy] += [ round_['SIG. STR. %']]          
                    fighters1totals[idy] += [ round_['TOTAL STR.']['attempted']]         
                    fighters1totals[idy] += [ round_['TOTAL STR.']['landed']]       
                    fighters1totals[idy] += [ round_['TD']['attempted']]       
                    fighters1totals[idy] += [ round_['TD']['landed']]     
                    fighters1totals[idy] += [ round_['TD %']]         
                    fighters1totals[idy] += [ round_['SUB. ATT']]       
                    fighters1totals[idy] += [ round_['PASS']]
                    fighters1totals[idy] += [ round_['REV.']]


                    bout_string += fighters1totals[idy]
                    fought_rounds = 1+idy
                    

                if idx == 1:
                    if idx == 1 and idy == 0:
                        for a in range(fought_rounds+1,6):
                            bout_string += ['']*12

                    fighters2totals[idy] += [ round_['KD']]           
                    fighters2totals[idy] += [ round_['SIG. STR.']['attempted']]         
                    fighters2totals[idy] += [ round_['SIG. STR.']['landed']]    
                    fighters2totals[idy] += [ round_['SIG. STR. %']]          
                    fighters2totals[idy] += [ round_['TOTAL STR.']['attempted']]         
                    fighters2totals[idy] += [ round_['TOTAL STR.']['landed']]       
                    fighters2totals[idy] += [ round_['TD']['attempted']]       
                    fighters2totals[idy] += [ round_['TD']['landed']]     
                    fighters2totals[idy] += [ round_['TD %']]         
                    fighters2totals[idy] += [ round_['SUB. ATT']]       
                    fighters2totals[idy] += [ round_['PASS']]
                    fighters2totals[idy] += [ round_['REV.']]                        

                    bout_string += fighters2totals[idy]

        if fought_rounds == 0:
            for a in range(fought_rounds+1,6):
                bout_string += ['']*24
        else:        
            for a in range(fought_rounds+1,6):
                bout_string += ['']*12

        
        Names += ["PpP b","Flw b","Bw b","Fw b","Lw b","Ww b","Mw b","LHw b","Hw b","WSw b","WFlw b","WBw b","WFw b"]
        Names += ["PpP a","Flw a","Bw a","Fw a","Lw a","Ww a","Mw a","LHw a","Hw a","WSw a","WFlw a","WBw a","WFw a"]

        if rank[0] or rank[1]:
            ranks_mod = get_weightclass_place(rank)
            bout_string+= ranks_mod[0]
            bout_string+= ranks_mod[1]
        else:
            bout_string+= ['']*26

        return [Names,bout_string]

    else:
        Names += ['UFC']
        bout_string += ['0']
        
        ufc_additional_names = ["fight bonus","perf bonus","sub bonus","ko bonus","method","round","title fight","details","time","time format","referee","f card1","op card1","f card2","op card2","f card2","op card2"]
        
        Names += ufc_additional_names

        totals = ["f KD","f SIG STR att","f SIG STR lan","f SIG STR %","f TOTAL STR att","f TOTAL STR lan","f TD att","f TD lan","f TD %","f SUB ATT","f PASS","f REV"]
        signifs = ["f SIG STR att","f SIG STR lan","f SIG STR %","f HEAD att","f HEAD lan","f BODY att","f BODY lan","f LEG att","f LEG lan","f DISTANCE att","f DISTANCE lan","f CLINCH att","f CLINCH lan","f GROUND att","f GROUND lan"]

        Names += totals
        Names += signifs

        Names += [x.replace('f','o') for x in totals]
        Names += [x.replace('f','o') for x in signifs]

        for x in range(1,6): 
            Names += ['sig ' + a + f' pr{x}' for a in signifs]

        for x in range(1,6): 
            Names += ['sig ' + a.replace('f','o') + f' pr{x}' for a in signifs]

        for x in range(1,6): 
            Names += ['tot ' + a + f' pr{x}' for a in totals]

        for x in range(1,6): 
            Names += ['tot ' + a.replace('f','o') + f' pr{x}' for a in totals]

        Names += ["PpP b","Flw b","Bw b","Fw b","Lw b","Ww b","Mw b","LHw b","Hw b","WSw b","WFlw b","WBw b","WFw b"]
        Names += ["PpP a","Flw a","Bw a","Fw a","Lw a","Ww a","Mw a","LHw a","Hw a","WSw a","WFlw a","WBw a","WFw a"]
        
        bout_string+= ['']*367
        
        return [Names,bout_string]

In [55]:
def get_fighter_lines(fighter):
    ufc_stat_prof = fighter[0]
    sher_stat_prof = fighter[1]
    tap_stat_prof = fighter[2]
    #sher, tap,ufcstat, date, dict: before, after
    bouts = fighter[3]
    
    name = ufc_stat_prof['Name']
    start_name, start_info = ufcstat_profile_to_string(ufc_stat_prof)
    
    sher_info = sher_profile_to_string(sher_stat_prof)
    start_name += sher_info[0]
    start_info += sher_info[1]
    
    tap_info = tap_profile_to_string(tap_stat_prof)
    start_name += tap_info[0]
    start_info += tap_info[1]
    
    bouts_list = [bout_decomposition(x,name) for x in bouts]
    for bout in bouts_list:
        bout[0] = start_name + bout[0]
        bout[1] = start_info + bout[1]
    
    #for x in bouts:

    return bouts_list   

In [56]:
import unidecode

In [57]:
def to_csv(everything,name):
    with open(name,'w') as table_file:
        fighter_writer = csv.writer(table_file,delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        head = []
        for fighter in everything:
            for line in get_fighter_lines(fighter):
                if not head:
                    head += line[0]
                    fighter_writer.writerow(head)
                fighter_writer.writerow([unidecode.unidecode(x) for x in line[1]])

In [58]:
to_csv(everything,'fighters_table_May_28.csv')